In [ ]:
import h5py
import os
import boto3
import shutil
import fire
import time
import torch
import transformers
import hydra
import numpy as np
import pandas as pd
from hydra.utils import get_original_cwd
from omegaconf import DictConfig
from datasets import load_dataset
import datasets
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler

In [ ]:
device = 'cuda'
local_path = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# GPT2 and OPT tokenizer test

In [ ]:
# GPT2

In [ ]:
#tokenizer GPT2
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',do_lower_case=False, truncation_side='left')

special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
    ]
}

tokenizer.add_special_tokens(special_tokens)
# tokenizer.pad_token_id = (
#     0  # unk. we want this to be different from the eos token
# )
# tokenizer.padding_side = "right"  # Left: Allows batched inference, we put right for this task.


In [ ]:
path_gpt2 = local_path + "/data/unsupervised.h5"

In [ ]:
with h5py.File(path_gpt2, 'r') as f:
    data_np = f['train'][:]
    train_dataset = torch.tensor(f['train'][:]).to(device)

In [ ]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=2)

In [ ]:
line = train_dataloader.dataset[0]

In [ ]:
tokenizer

In [ ]:
tokenizer.decode(line, clean_up_tokenization_spaces=True)

In [ ]:
#tokenizer OPT
model_path = local_path + "/checkpoints/opt/checkpoint-opt-final"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, do_lower_case=False, truncation_side='left')
max_token_len = tokenizer.max_model_input_sizes["gpt2"]

special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
    ]
}

tokenizer.add_special_tokens(special_tokens)

In [ ]:
path_opt = local_path + "/data/unsupervised_opt.h5"

In [ ]:
with h5py.File(path_opt, 'r') as f:
    data_np = f['train'][:]
    train_dataset = torch.tensor(f['train'][:]).to(device)

In [ ]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=2)

In [ ]:
line = train_dataloader.dataset[0]

In [ ]:
tokenizer.decode(line, clean_up_tokenization_spaces=True)

# Test Llama tokenizer and h5 dataset to use huggingface Trainer

In [ ]:
#tokenizer Llama
from transformers import LlamaForCausalLM, LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained(
    'decapoda-research/llama-7b-hf',
    do_lower_case=False,
    truncation_side='left'
)

In [ ]:
special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
    ]
}

tokenizer.add_special_tokens(special_tokens)
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "right"  # Left: Allows batched inference, we put right for this task.

In [ ]:
local_path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
path = local_path + "/data/unsupervised_llama.h5"

In [ ]:
with h5py.File(path, 'r') as f:
    print(f.keys())
    raw_data = f['train']
    train_dataset = torch.tensor(f['train'][:])

In [ ]:
train_dataset.shape

In [ ]:
raw_data

In [ ]:
datta = datasets.Dataset({'train':train_dataset})

In [ ]:
dset = datasets.DatasetDict({'train':train_dataset}, features=['input', 'label'])

In [ ]:
dset.features

In [ ]:
dset_dict = {
    'train': [train_dataset]
}

In [ ]:
px = pd.DataFrame(train_dataset)

In [ ]:
px

In [ ]:
dset = load_dataset(raw_data)

In [ ]:
train_sampler = RandomSampler(train_dataset)

In [ ]:
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=2)

In [ ]:
train_dataloader.dataset[0]

In [ ]:
line = train_dataloader.dataset[0]

In [ ]:
tokenizer.decode(line, clean_up_tokenization_spaces=True)

In [ ]:
data_np.shape

In [ ]:
dataset = datasets.Dataset.from_pandas(train_dataset)

In [ ]:
data_files = {"train": byte}

In [ ]:
train_dataloader

In [ ]:
import datasets
from datasets import Dataset
import pandas as pd
df_train = pd.read_hdf(path)
sentences = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train)
    }
)